## datapreprocessing with question set

In [41]:
import pandas as pd
import numpy as np
import re
from nltk.stem import SnowballStemmer

In [37]:
df = pd.read_csv("C:\\Users\\DU357159\\Documents\\ENU_DATA\questions.csv")

In [43]:
def review_to_wordlist(review, remove_stopwords=True):
    # Clean the text, with the option to remove stopwords.
    
    # Convert words to lower case and split them
    words = str(review).lower().split()

    # Optionally remove stop words (true by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    
    review_text = " ".join(words)

    # Clean the text
    review_text = re.sub(r"[^A-Za-z0-9(),!.?\'\`]", " ", review_text)
    review_text = re.sub(r"\'s", " 's ", review_text)
    review_text = re.sub(r"\'ve", " 've ", review_text)
    review_text = re.sub(r"n\'t", " 't ", review_text)
    review_text = re.sub(r"\'re", " 're ", review_text)
    review_text = re.sub(r"\'d", " 'd ", review_text)
    review_text = re.sub(r"\'ll", " 'll ", review_text)
    review_text = re.sub(r",", " ", review_text)
    review_text = re.sub(r"\.", " ", review_text)
    review_text = re.sub(r"!", " ", review_text)
    review_text = re.sub(r"\(", " ( ", review_text)
    review_text = re.sub(r"\)", " ) ", review_text)
    review_text = re.sub(r"\?", " ", review_text)
    review_text = re.sub(r"\s{2,}", " ", review_text)
    
    words = review_text.split()
    
    # Shorten words to their stems
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in words]
    
    review_text = " ".join(stemmed_words)
    
    # Return a list of words
    return(review_text)

In [44]:
def process_questions(question_list, questions, question_list_name):
# function to transform questions and display progress
    for question in questions:
        question_list.append(review_to_wordlist(question))
        ##if len(question_list) % 10000 == 0:
            ##progress = len(question_list)/len(df) * 100
           ##print("{} is {}% complete.".format(question_list_name, round(progress, 1)))

In [45]:
questions1 = []     
process_questions(questions1, df.question1, "questions1")
print()
questions2 = []     
process_questions(questions2, df.question2, "questions2")

In [49]:
invalid_questions1 = []
invalid_questions2 = []
for i in range(len(questions1)):
    # questions need to contain a vowel (which should be part of a full word) to be valid
    if not re.search('[aeiouyAEIOUY]', questions1[i]):
    # Need to subtract 'len(invalid_questions)' to adjust for the changing index values as questions are removed.
        invalid_questions1.append(questions1[i])
    if not re.search('[aeiouyAEIOUY]', questions2[i]):
    # Need to subtract 'len(invalid_questions)' to adjust for the changing index values as questions are removed.
        invalid_questions2.append(questions2[i])


In [54]:
question1 = [question for question in questions1 if question not in invalid_questions1]
question2 = [question for question in questions2 if question not in invalid_questions2]

In [63]:
question1[1]

'stori kohinoor ( koh i noor ) diamond'

In [70]:
q1 = [w for w in question1]
q2 = []
for i in q1:
    q2.extend(i.split())

## data preprocessing

In [19]:
import nltk
from nltk.corpus import stopwords

In [20]:
stop_words_en = stopwords.words("english")
def preprocessing(raw):
    wordlist = nltk.word_tokenize(raw)
    text = [w for w in wordlist if w not in stop_words_en]
    return text
f1 = open("C:\\Users\\DU357159\\Documents\\ENU_DATA\\data\\SampleTextFile_500kb.txt" , "r" ,encoding = "utf8")
text1 = preprocessing(f1.read())
f2 = open("C:\\Users\\DU357159\\Documents\\ENU_DATA\\data\\SampleTextFile_1000kb.txt" , "r" ,encoding = "utf8")
text2 = preprocessing(f2.read())


In [21]:
import gensim
from gensim.models.doc2vec import TaggedDocument

In [73]:
taggeddocs = []
doc1 = TaggedDocument(words = text1 , tags = ["text1"])
taggeddocs.append(doc1)
doc2 = TaggedDocument(words = q2 , tags = ["text1"])
taggeddocs.append(doc2)

In [74]:
model = gensim.models.Doc2Vec(taggeddocs ,dm = 0,alpha = 0.025 , size =20 , min_alpha = 0.025 ,min_count = 0)

C:\Users\DU357159\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\models\doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [75]:
model.train(taggeddocs ,epochs = 20 , total_examples=model.corpus_count)

In [76]:
similarity = model.n_similarity(text1,q2)
print("similarity index : {:4.2f}%".format(similarity*100))

C:\Users\DU357159\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.


similarity index : 6.09%
